In [1]:
import pandas as pd

# Combine DE results with old functional annotation mega table

In [2]:
excel_file = '/depot/jwisecav/data/ceratopteris/her7/02_functional_annotations/Ceratopteris_mega_table.xls'  # Replace with the path to your Excel file
outfile = '/depot/jwisecav/data/ceratopteris/her7/02_functional_annotations/Ceratopteris_DEG_annotations.txt'  # Replace with the desired path for the output file

infiles = ['/depot/jwisecav/data/ceratopteris/her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_her19_v_herm_de_results.txt',
          '/depot/jwisecav/data/ceratopteris/her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_male_v_her19_de_results.txt',
          '/depot/jwisecav/data/ceratopteris/her7/10_gene_exp/gene_expression_jen_local/10_gene_exp/burow_male_v_herm_de_results.txt']

In [3]:
# Load your Excel file into a pandas DataFrame
df1 = pd.read_excel(excel_file)

merged_df = df1

for infile in infiles:
    comparison = infile.split('/burow_')[1].split('_de_result')[0]
    print(comparison)
    
    df2 = pd.read_csv(infile, sep='\t')
    df2.index.name = 'New Gene ID'
    df2 = df2.drop(columns=['baseMean', 'lfcSE', 'pvalue'])
    df2 = df2.rename(columns={'log2FoldChange': comparison + '_LFC'})
    df2 = df2.rename(columns={'padj': comparison + '_padj'})
    
    merged_df = pd.merge(merged_df, df2, on='New Gene ID', how='left')
    
merged_df.to_csv(outfile, sep='\t', index=False, header=True)

her19_v_herm
male_v_her19
male_v_herm


# Add Arabidopsis homologs to table

In [36]:
excel_file = '/depot/jwisecav/data/ceratopteris/her7/02_functional_annotations/Ceratopteris_DEG_annotations.xls'  # Replace with the path to your Excel file
symfile = '/depot/jwisecav/data/ceratopteris/her7/02_functional_annotations/Arabidopsis_gene_symbols.txt'
ogfile = '/depot/jwisecav/data/ceratopteris/fern_phylogenetics/2_orthofinder/Phylogenetic_Hierarchical_Orthogroups/N5.tsv'
outfile = '/depot/jwisecav/data/ceratopteris/her7/02_functional_annotations/Ceratopteris_DEG_annotations_w_Athomologs.txt'  # Replace with the desired path for the output file


In [19]:
fi = open(symfile)

symdict = {}
for line in fi:
    
    symbol, gene = line.rstrip().split('\t')
    #print(symbol,gene)
    symdict[gene] = symbol

fi.close()

In [27]:
columns = ['New Gene ID', 'Arabidopsis homologs']
df2 = pd.DataFrame(columns=columns)

fi = open(ogfile)

for line in fi:
    col = line.rstrip().split('\t')
    if col[0] == 'HOG':
        continue

    ageneset = set()

    if len(col) < 9:
        continue

    agenes = col[8].split(', ')
    if agenes == ['']:
        continue

    for agene in agenes:
        agene = agene.split('.')[0]
        if agene in symdict:
            symbol = symdict[agene]
            #print(agene, symbol)
            agene = symbol + ' (' + agene + ')'
        #print(agene)
        ageneset.add(agene)
        
    if len(col) < 12:
        continue
    
    cgenes = col[11].split(', ')
    if cgenes == ['']:
        continue
        
    for cgene in cgenes:
        cgene = 'Ceric.' + cgene.split('.')[1]
        homologs = '; '.join(ageneset)
        #print(cgene, homologs)
        df2 = df2.append({'New Gene ID': cgene, 'Arabidopsis homologs': homologs}, ignore_index=True)

fi.close()

In [28]:
df2

,New Gene ID,Arabidopsis homologs
0,Ceric.11G030800,AT4G13620
1,Ceric.28G014900,AT4G13620
2,Ceric.10G005000,AT4G13620
3,Ceric.06G023400,AT4G13620
4,Ceric.20G089300,AT1G28160; LEP (AT5G13910); PUCHI (AT5G18560)
...,...,...
12456,Ceric.32G066600,AT2G28920
12457,Ceric.10G089900,AT2G28920
12458,Ceric.1Z082600,AT3G45920; AT3G46355; AT3G46410
12459,Ceric.35G010300,AT4G15885


In [37]:
# Load your Excel file into a pandas DataFrame
df1 = pd.read_excel(excel_file)
merged_df = pd.merge(df2, df1, on='New Gene ID', how='right')
merged_df.to_csv(outfile, sep='\t', index=False, header=True)

In [32]:
merged_df

,New Gene ID,Arabidopsis homologs,her19_v_herm LFC,her19_v_herm padj,male_v_her19 LFC,male_v_her19 padj,male_v_herm LFC,male_v_herm padj,mean Her1 counts,mean Her19 counts,mean Her3 counts,mean male counts,mean sporophyte counts,EC,GO,KEGG,KOG,PANTHER,PFAM,SIGNALP
0,Ceric.37G029400,NaN,0.025935,NaN,1.986296,1.245008e-01,1.504412,1.540904e-01,0.000000,0.000000,1.212733e+02,0.000000,1.106112e+03,NaN,NaN,NaN,NaN,NaN,NaN,SignalP-TM
1,Ceric.29G072800,NaN,-0.193360,1.000000e+00,0.558558,NaN,-0.148950,9.999965e-01,0.000000,42.513333,5.276333e+01,8.470000,4.281400e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ceric.1Z302300,NaN,-0.079909,NaN,0.000000,NaN,-0.219401,NaN,0.000000,55.440000,5.234667e+01,0.000000,4.685100e+02,"2.7.11.1, Non-specific serine/threonine protei...",NaN,NaN,NaN,"PTHR11584, SERINE/THREONINE PROTEIN KINASE; PT...",NaN,NaN
3,Ceric.38G027400,NaN,-0.123072,1.000000e+00,-0.803201,4.854079e-01,-1.399572,1.991500e-01,0.000000,55.046667,5.141333e+01,28.576667,4.159660e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ceric.10G069500,NaN,-3.829494,8.617684e-33,9.724716,6.093886e-278,5.886575,1.345708e-120,9035.983333,5537.710000,2.156867e+04,199168.373333,6.204000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34934,Ceric.09G050600,NaN,0.146118,1.000000e+00,-3.036895,8.867012e-38,-2.731764,6.469359e-23,250453.283333,179362.853333,2.204445e+05,78205.613333,2.072460e+05,"4.1.1.39, Ribulose-bisphosphate carboxylase.",NaN,"K01602, ribulose-bisphosphate carboxylase smal...",NaN,"PTHR31262, FAMILY NOT NAMED","PF00101, Ribulose bisphosphate carboxylase, sm...",NaN
34935,Ceric.1Z328500,NaN,-0.391581,9.548357e-01,0.873297,3.830282e-01,0.035763,9.999965e-01,280090.366667,368766.716667,4.551960e+05,335342.153333,1.501240e+07,NaN,NaN,NaN,NaN,PTHR33047,NaN,NaN
34936,Ceric.23G004900,NaN,0.028175,1.000000e+00,-0.173337,9.999995e-01,-0.082801,9.999965e-01,287988.216667,480945.253333,2.822100e+05,535421.686667,2.349515e+05,NaN,"GO:0046872, metal ion binding; GO:0030001, met...","K07213, copper chaperone","KOG1603, Copper chaperone","PTHR22814:SF172; PTHR22814, COPPER TRANSPORT P...","PF00403, Heavy-metal-associated domain",NaN
34937,Ceric.16G028200,NaN,-0.126039,1.000000e+00,-2.229834,3.153188e-08,-2.526958,1.132019e-08,420998.100000,945653.316667,4.762145e+05,187560.310000,1.060704e+05,NaN,NaN,NaN,NaN,NaN,NaN,SignalP-noTM
